## 2. Irrigation Insight Tool

This script uses the wetlands insight tool to query the polygons from the irrigated area analysis. The input is the processed shapefile from running 'preprocess_irrigatedWIT.ipynb', the output is a .csv file for every unique polygon in the input shapefile.

Make sure to create the output directory before running the script. The code is multiprocessed.

### User Inputs

In [7]:
shp_path = 'data/shapes_WIT_ready/GWYDIR_2018_WIT.shp'
year = '2018'
ncpus = 6
Output_dir = 'results/GWYDIR/2018/csvs/'
columnName= 'UniqueID'

#### Run the cells below to generate a csv for each polygon in the shapefile

In [8]:
from datacube.utils import geometry
import fiona
from multiprocessing import Pool
import warnings
import sys

sys.path.append('src/')
from WITdrill import WITdrill

warnings.filterwarnings("ignore")

In [9]:
#get time period for function
start = year+'-06-01'
end = str(int(year)+1)+'-06-01'
time_period = (start, end)

In [10]:
with fiona.open(shp_path) as input:
    crs = geometry.CRS(input.crs_wkt)

In [11]:
p = Pool(ncpus)
for feat in fiona.open(shp_path):
#     WITdrill(feat,crs, time_period, Output_dir, columnName) # single-cpu
    p.apply_async(WITdrill, [feat, crs, time_period, Output_dir, columnName]) # MULTIPROCESS

### Converting shapes to geojson for uploading on Terria-Cube

In [13]:
import geopandas as gpd
shp_path = 'data/shapes_WIT_ready/GWYDIR_2018_WIT.shp'

x = gpd.read_file(shp_path)
x84 = x.to_crs({'init': 'epsg:4326'})
x84.to_file("results/GWYDIR/GWYDIR_2018_WIT.geojson", driver='GeoJSON')